In [100]:
import json
import jsonlines
import random

# 定义从文件中读取 JSON 数据的函数
def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# 定义将结果写入文件的函数
def write_json_file(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
        
# 假设输入文件路径和输出文件路径
input_file = 'hotpot_train_v1.1.json'  # 本地 JSON 文件路径
output_file = 'train.json'  # 输出文件路径
# /home/lpc/repos/HotPotQA/ChatGLM_PEFT/main/data/train.json

In [101]:

data_list= read_json_file(input_file)

for data in data_list:
    if 'type' in data:
        del data['type']    
    if 'level' in data:
        del data['level']  
    #去除超出index范围的sp 
    valid_supporting_facts = []
    context = data["context"]
    # print(context[0][0])
    context = {item[0]: item[1] for item in context}
    # print(context)
    # sp[0] :title 在context中是唯一标识符
    for sp in data["supporting_facts"]:
        if(sp[1]<len(context[sp[0]])):
            valid_supporting_facts.append(sp)
    # print(len(valid_supporting_facts))
    sp_map = []
    for sp in valid_supporting_facts:
        if sp[0] not in sp_map:
            sp_map.append(sp[0])
    data["supporting_facts"] = valid_supporting_facts


    # 剪除过多的context段落
    lenth = len(sp_map)

    # 选取有效和无效 1:1 所以只有<5的context才做剪除

    if lenth < 5:
        need_num = 0
        context = []
        for item in data["context"]:
            if item[0] in sp_map:
                context.append(item)
            elif need_num < lenth:
                context.append(item)
                need_num += 1
        random.shuffle(context)
        data["context"] = context
        #     print(item)
        #     if item[0] not in sp_map and remove_num >= 0:
        #         # print(item[0])
        #         data["context"].remove(item)
        #         # print("context_len:"+ str(len(data["context"]))+"\n")
        #         remove_num -= 1
        #         # print("remove_num:"+ str(remove_num)+"\n")
        # print("\n")
        # random.shuffle(data["context"])
        # for item in data["context"]:
        #     print(item[0])

# 把sp中的index换成原句，LLM对数字理解能力不强，难以定位到index
for data in data_list:
    context = data["context"]
    context = {item[0]: item[1] for item in context}
    for sp in data["supporting_facts"]:
        if(sp[1]<len(context[sp[0]])):
            sp[1] = context[sp[0]][sp[1]]
        # print(sp[1])
        # print("\n")
        # print(result[sp[0]][sp[1]])
        # print("\n")

# print(data_list[0]["context"]["Ed Wood (film)"][1])
# context = data[0]["context"]
# result = {item[0]: item[1] for item in context}
# print(result["Ed Wood (film)"][1])
# print(data_list[0]["supporting_facts"])





In [102]:
# 定义函数来处理每个对象并生成所需格式的列表
def process_data(data_list):
    output = []
    
    for data in data_list:
        fix_object = {
            "conversations" :[]
        }
        # 生成问题对象
        question_object = {
            "role": "user",
            "content": {
                "question": data["question"],
                "context": data["context"]
            }
        }
        question_object["content"] = json.dumps(question_object["content"], ensure_ascii=False)
        # output.append(question_object)
        fix_object["conversations"].append(question_object)
        # 生成回答对象
        answer_object = {
            "role": "assistant",
            "content": {
                "answer": data["answer"],
                "supporting_facts": data["supporting_facts"]
            }
        }
        answer_object["content"]  = json.dumps(answer_object["content"], ensure_ascii=False)
        # output.append(answer_object)
        fix_object["conversations"].append(answer_object)
        output.append(fix_object)
    return output

In [103]:
output_data = process_data(data_list)

print(output_data[0])
with jsonlines.open(output_file, 'w') as writer:    
    for item in output_data:
        writer.write(item)


{'conversations': [{'role': 'user', 'content': '{"question": "Which magazine was started first Arthur\'s Magazine or First for Women?", "context": [["History of Albanian football", ["Football in Albania existed before the Albanian Football Federation (FSHF) was created.", " This was evidenced by the team\'s registration at the Balkan Cup tournament during 1929-1931, which started in 1929 (although Albania eventually had pressure from the teams because of competition, competition started first and was strong enough in the duels) .", " Albanian National Team was founded on June 6, 1930, but Albania had to wait 16 years to play its first international match and then defeated Yugoslavia in 1946.", " In 1932, Albania joined FIFA (during the 12–16 June convention ) And in 1954 she was one of the founding members of UEFA."]], ["Arthur\'s Magazine", ["Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.", " Edited by T.S. Arthur, it 

In [104]:
# 将结果写入到输出文件中
# output_data =  json.dumps(output_data, ensure_ascii=False)
# write_json_file(output_file, output_data)
# with jsonlines.open(output_file, 'w') as writer:
#     writer.write(output_data)


In [105]:
# 读取输入
# input = read_json_file("output.json")
# str = json.dumps(input, ensure_ascii=False)
# print(str)